### Cohort Retention Analysis
Cohort : 유사한 특성 (가입기간, 연령, 성별 등) 을 가진 집단<br>
Retention : AARRR의 'Retention' 단계는 사용자들이 서비스에 **다시** 방문하는가를 수치화하는 지표

즉, Cohort Retention이란 어떠한 특성을 가진 사용자 집단의 재방문 추이를 살펴볼 수 있는 분석 기법이다.

In [89]:
import pymysql
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import csv
import os
import time
from dotenv import load_dotenv
load_dotenv()
MYSQL_ROOT_PW = os.environ.get("MYSQL_ROOT_PW")
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [90]:
con = pymysql.connect(host="localhost", db="ecommerce", user="root", password=MYSQL_ROOT_PW)
cur = con.cursor()

In [91]:
def fetchresult_to_df(query):
    cur.execute(query)
    result = cur.fetchall()
    return pd.DataFrame(result)

In [92]:
try:
    query = """
    SELECT *
    FROM retail
    LIMIT 3;
    """
    df = fetchresult_to_df(query)
    df.columns = ["invoice_no", "stock_code", "description", "quantity",
                    "invoice_date", "unit_price", "customer_id", "country"]
    display(df)
except:
    print("SQL Server Connection Error!")

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850,United Kingdom


In [94]:
# Retention chart를 생성하기 위한 column만을 추출한 View를 구성
# year + month => 월별 최초 구매가 발생한 customer의 이후 retention을 살펴보기 위함이다.
# YEAR(invoice_date), MONTH(invoice_date) 하지 않는 이유는, 각 년월(특히 월)의 format을 zero-padding 된 string으로 유지하기 위함이다.
# e.g.) "2022-03-08"을  MONTH(invoice_date) 하면 '3', SUBSTRING(invoice_date, 6, 2) 하면 '03'

try:
    query = """
    CREATE VIEW sales as 
    (
        SELECT substring(invoice_date, 1, 4) year,
             substring(invoice_date, 6, 2) month,
            customer_id,
            country,
            quantity
        FROM retail
    );
    """
    cur.execute(query)
except:
    print("SQL Server Connection Error!")

In [95]:
try:
    query = """
    SELECT *
    FROM SALES;
    """
    df = fetchresult_to_df(query)
    df.columns = ["year", "month", "customer_id", "country", "quantity"]
    display(df.head(3))
except:
    print("SQL Server Connection Error!")

,year,month,customer_id,country,quantity
0,2010,12,17850,United Kingdom,6
1,2010,12,17850,United Kingdom,6
2,2010,12,17850,United Kingdom,8


#### 1. Retention 계산을 위한 Year-Month Index 생성

특정 코호트로 분류된 사용자의 m+1, m+2, ... 구매 이력을 분류하기 위하여
시작년월 ~ 다음년월 간의 관계를 나타내는 index가 포함된 Base Year-Month View를 생성한다.

e.g.)

| customer | first_year | first_month | next_year | next_month | index (m+idx) |
|----------|------------|-------------|-----------|------------|---------------|
| Noel     | 2001       | 10          | 2001      | 12         | 2             |
| Noel     | 2002       | 02          | 2002      | 05         | 3             |
| Liam     | 2002      | 02          | 2002      | 07         | 5             |

In [96]:
try:
    query = """
    CREATE VIEW monthly_idx as
    (
        WITH distinct_ym as 
        (
            SELECT year, month
            FROM sales
            GROUP BY 1, 2
        )
        
        SELECT *,
            ROW_NUMBER() OVER(PARTITION BY first_ym order by next_ym) idx
        FROM (SELECT year first_year, month first_month, concat(year, month) first_ym from distinct_ym) A
            CROSS JOIN
            (SELECT year next_year, month next_month, concat(year, month) next_ym from distinct_ym) B
        WHERE first_ym <= next_ym
    );
    """
    cur.execute(query)
except:
    print("SQL Server Connection Error!")

In [97]:
try:
    query = """
    SELECT *
    FROM monthly_idx
    """
    df = fetchresult_to_df(query)
    df.columns = ["first_year", "first_month", "first_ym", "next_year", "next_month", "next_ym", "index"]
    display(df.head(5))
except:
    print("SQL Server Connection Error!")

,first_year,first_month,first_ym,next_year,next_month,next_ym,index
0,2010,12,201012,2010,12,201012,1
1,2010,12,201012,2011,01,201101,2
2,2010,12,201012,2011,02,201102,3
3,2010,12,201012,2011,03,201103,4
4,2010,12,201012,2011,04,201104,5


In [99]:
try:
    query = """
    CREATE VIEW sales_fin as
    (
        WITH first_CTE as (
            SELECT customer_id, min(CONCAT(year, month)) first_ym
            FROM sales
            GROUP BY 1
        )

        SELECT customer_id, year, month, country, quantity, idx
        FROM sales S
            JOIN first_CTE F USING (customer_id)
            JOIN monthly_idx M 
                on F.first_ym = M.first_ym
                    and concat(S.year, S.month) = next_ym
    );
    """
    cur.execute(query)
except:
    print("SQL Server Connection Error!")

In [102]:
try:
    query = """
    SELECT *
    FROM sales_fin
    LIMIT 5;
    """
    df = fetchresult_to_df(query)
    df.columns = ["cutomer_id", "year", "month", "country", "quantity", "Index"]
    display(df)
except:
    print("SQL Server Connection Error!")

,cutomer_id,year,month,country,quantity,Index
0,17850,2010,12,United Kingdom,6,1
1,17850,2010,12,United Kingdom,6,1
2,17850,2010,12,United Kingdom,8,1
3,17850,2010,12,United Kingdom,6,1
4,17850,2010,12,United Kingdom,6,1


#### 2. Retention 분석